In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies_data_1 = pd.read_csv('/content/drive/MyDrive/tmdb_5000_movies.csv')
movies_data_2 = pd.read_csv('/content/drive/MyDrive/tmdb_5000_credits.csv')

In [3]:
movies_data_1.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
movies_data_2.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [5]:
print(movies_data_1.shape,movies_data_2.shape)

(4803, 20) (4803, 4)


In [6]:
movies_data_1.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [7]:
movies_data_2.columns

Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')

In [8]:
final_data = pd.merge(movies_data_1, movies_data_2, left_on='id', right_on='movie_id', how='inner')

In [9]:
final_data['index'] = range(0, len(final_data))

In [10]:
final_data.isnull().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title_x                    0
vote_average               0
vote_count                 0
movie_id                   0
title_y                    0
cast                       0
crew                       0
index                      0
dtype: int64

In [11]:
final_data.drop('homepage', axis=1, inplace=True)

In [12]:
import json
def extract_names(json_str):
    if isinstance(json_str, list): # Check if the input is already a list
        return [item['name'] for item in json_str if isinstance(item, dict) and 'name' in item]
    try:
        return [item['name'] for item in json.loads(json_str)]
    except (json.JSONDecodeError, TypeError): # Handle cases where json.loads might fail
        return []

final_data['genre_names'] = final_data['genres'].apply(extract_names)
final_data['keyword_names'] = final_data['keywords'].apply(extract_names)
final_data['company_names'] = final_data['production_companies'].apply(extract_names)
final_data['cast_names'] = final_data['cast'].apply(extract_names)

# Print the modified DataFrame
print(final_data[['id', 'genre_names', 'keyword_names', 'company_names', 'cast_names']])

          id                                    genre_names  \
0      19995  [Action, Adventure, Fantasy, Science Fiction]   
1        285                   [Adventure, Fantasy, Action]   
2     206647                     [Action, Adventure, Crime]   
3      49026               [Action, Crime, Drama, Thriller]   
4      49529           [Action, Adventure, Science Fiction]   
...      ...                                            ...   
4798    9367                      [Action, Crime, Thriller]   
4799   72766                              [Comedy, Romance]   
4800  231617             [Comedy, Drama, Romance, TV Movie]   
4801  126186                                             []   
4802   25975                                  [Documentary]   

                                          keyword_names  \
0     [culture clash, future, space war, space colon...   
1     [ocean, drug abuse, exotic island, east india ...   
2     [spy, based on novel, secret agent, sequel, mi...   
3     [

In [13]:
final_data.head()

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,...,vote_count,movie_id,title_y,cast,crew,index,genre_names,keyword_names,company_names,cast_names
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Ingenious Film Partners, Twentieth Century Fo...","[Sam Worthington, Zoe Saldana, Sigourney Weave..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Walt Disney Pictures, Jerry Bruckheimer Films...","[Johnny Depp, Orlando Bloom, Keira Knightley, ..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",...,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Columbia Pictures, Danjaq, B24]","[Daniel Craig, Christoph Waltz, Léa Seydoux, R..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Legendary Pictures, Warner Bros., DC Entertai...","[Christian Bale, Michael Caine, Gary Oldman, A..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",[Walt Disney Pictures],"[Taylor Kitsch, Lynn Collins, Samantha Morton,..."


In [14]:
selected_features = ['original_language','original_title','overview','genre_names','keyword_names','company_names','cast_names']

In [15]:
for feature in selected_features:
  final_data[feature] = final_data[feature].fillna('')

In [16]:
final_data.dtypes

budget                    int64
genres                   object
id                        int64
keywords                 object
original_language        object
original_title           object
overview                 object
popularity              float64
production_companies     object
production_countries     object
release_date             object
revenue                   int64
runtime                 float64
spoken_languages         object
status                   object
tagline                  object
title_x                  object
vote_average            float64
vote_count                int64
movie_id                  int64
title_y                  object
cast                     object
crew                     object
index                     int64
genre_names              object
keyword_names            object
company_names            object
cast_names               object
dtype: object

In [17]:
final_data['original_language'] = final_data['original_language'].astype(str)
final_data['original_title'] = final_data['original_title'].astype(str)
final_data['overview'] = final_data['overview'].astype(str)

# Convert JSON strings to lists if needed
def convert_json_to_list(column):
    return column.apply(lambda x: json.loads(x) if isinstance(x, str) else x)

final_data['genre_names'] = convert_json_to_list(final_data['genre_names'])
final_data['keyword_names'] = convert_json_to_list(final_data['keyword_names'])
final_data['company_names'] = convert_json_to_list(final_data['company_names'])
final_data['cast_names'] = convert_json_to_list(final_data['cast_names'])

# Define the function to combine features
def combine_features(row):
    return ' '.join([
        str(row['original_language']),
        row['original_title'],
        row['overview'],
        ' '.join(row['genre_names']) if isinstance(row['genre_names'], list) else '',
        ' '.join(row['keyword_names']) if isinstance(row['keyword_names'], list) else '',
        ' '.join(row['company_names']) if isinstance(row['company_names'], list) else '',
        ' '.join(row['cast_names']) if isinstance(row['cast_names'], list) else ''
    ])

# Apply the function to create the combined_features column
final_data['combined_features'] = final_data.apply(combine_features, axis=1)

# Print the DataFrame with the new column
print(final_data[['id', 'combined_features']])

          id                                  combined_features
0      19995  en Avatar In the 22nd century, a paraplegic Ma...
1        285  en Pirates of the Caribbean: At World's End Ca...
2     206647  en Spectre A cryptic message from Bond’s past ...
3      49026  en The Dark Knight Rises Following the death o...
4      49529  en John Carter John Carter is a war-weary, for...
...      ...                                                ...
4798    9367  es El Mariachi El Mariachi just wants to play ...
4799   72766  en Newlyweds A newlywed couple's honeymoon is ...
4800  231617  en Signed, Sealed, Delivered "Signed, Sealed, ...
4801  126186  en Shanghai Calling When ambitious New York at...
4802   25975  en My Date with Drew Ever since the second gra...

[4803 rows x 2 columns]


In [18]:
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(final_data['combined_features'])

In [19]:
print(feature_vectors)

  (0, 42441)	0.06068348004761908
  (0, 26384)	0.0791320729883438
  (0, 56387)	0.09044377596042182
  (0, 36410)	0.09044377596042182
  (0, 55053)	0.09044377596042182
  (0, 42904)	0.02969633383014744
  (0, 3962)	0.0527224418843608
  (0, 53514)	0.08330688599177509
  (0, 1537)	0.054759326117188284
  (0, 48796)	0.05398479901025534
  (0, 46163)	0.07289108311279675
  (0, 42837)	0.08626896295699049
  (0, 43833)	0.07289108311279675
  (0, 19602)	0.06969761762866993
  (0, 41488)	0.08626896295699049
  (0, 39983)	0.07754488274740465
  (0, 49029)	0.04890579910783227
  (0, 39644)	0.09044377596042182
  (0, 30438)	0.05398479901025534
  (0, 38244)	0.09044377596042182
  (0, 26602)	0.06443797527242737
  (0, 36714)	0.07616999602312839
  (0, 50730)	0.04482250986938652
  (0, 23955)	0.047462799528118244
  (0, 33955)	0.06969761762866993
  :	:
  (4802, 22475)	0.1146354043511805
  (4802, 54997)	0.03887067012160393
  (4802, 23193)	0.030910791902674272
  (4802, 16548)	0.06333490525174078
  (4802, 25416)	0.067075805

In [20]:
similarity = cosine_similarity(feature_vectors)

In [21]:
print(similarity)

[[1.         0.02282766 0.0131802  ... 0.01151266 0.02240945 0.00933639]
 [0.02282766 1.         0.0217799  ... 0.0253768  0.02553932 0.02359701]
 [0.0131802  0.0217799  1.         ... 0.01010255 0.00759066 0.00503299]
 ...
 [0.01151266 0.0253768  0.01010255 ... 1.         0.03901574 0.04082108]
 [0.02240945 0.02553932 0.00759066 ... 0.03901574 1.         0.03723769]
 [0.00933639 0.02359701 0.00503299 ... 0.04082108 0.03723769 1.        ]]


In [22]:
print(similarity.shape)

(4803, 4803)


In [23]:
movie_name = input('Enter your Favourite Movie : ')

Enter your Favourite Movie : Avengers


In [24]:
list_of_all_titles = sorted(final_data['original_title'].tolist())
print(list_of_all_titles)

['#Horror', '$upercapitalist', '(500) Days of Summer', "...E tu vivrai nel terrore! L'aldilà", '10 Cloverfield Lane', '10 Days in a Madhouse', '10 Things I Hate About You', '102 Dalmatians', '10th & Wolf', '11:14', '12 Angry Men', '12 Rounds', '12 Years a Slave', '127 Hours', '13 Going on 30', '13 Hours: The Secret Soldiers of Benghazi', '1408', '15 Minutes', '16 Blocks', '16 to Life', '17 Again', '1776', '1941', '1947: Earth', '1982', '2 Fast 2 Furious', '2 Guns', '20 Dates', '20 Feet from Stardom', '20,000 Leagues Under the Sea', '200 Cigarettes', '2001: A Space Odyssey', '2012', "2016: Obama's America", '2046', '21', '21 & Over', '21 Grams', '21 Jump Street', '22 Jump Street', '24 7: Twenty Four Seven', '25th Hour', '27 Dresses', '28 Days', '28 Days Later', '28 Weeks Later', '2:13', '3 Backyards', '3 Days to Kill', '3 Ninjas Kick Back', '3 Strikes', '30 Days of Night', '30 Minutes or Less', '30 Nights of Paranormal Activity With the Devil Inside the Girl With the Dragon Tattoo', '30

In [25]:
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['The Avengers', 'Swingers']


In [26]:
close_match = find_close_match[0]
print(close_match)

The Avengers


In [27]:
index_of_the_movie = final_data[final_data['original_title'] == close_match]['index'].values[0]
print(index_of_the_movie)

16


In [ ]:
similarity_score = list(enumerate(similarity[index_of_the_movie]))
len(similarity_score)

In [29]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(16, 0.9999999999999994), (7, 0.1932564152365516), (85, 0.17107278262038164), (129, 0.16341288370167775), (79, 0.13412945786690048), (26, 0.13159447175350295), (126, 0.13039404999203202), (31, 0.12605835928878192), (68, 0.11959143351148808), (511, 0.11812534718197491), (174, 0.11705287921233751), (169, 0.11683830716044948), (182, 0.10753945628016733), (3, 0.10648877421765612), (94, 0.10301760595305855), (813, 0.09960845783936319), (101, 0.09513793084663787), (242, 0.09357793552815276), (165, 0.09192628653371888), (38, 0.09102667062411887), (14, 0.08961729728023715), (5, 0.08853113198776556), (159, 0.08816614909831744), (215, 0.0870913407243048), (46, 0.08569848677816393), (1428, 0.0843135063514359), (64, 0.0830991970539487), (158, 0.08215222474109474), (30, 0.08126648904431734), (9, 0.08101591268900417), (218, 0.08096054137861874), (883, 0.07797870021123268), (203, 0.0779113822870687), (788, 0.0768133327218221), (4216, 0.07522688801758072), (1359, 0.07479124064093495), (1296, 0.074458

In [30]:
print('Movies suggested for you : \n')
i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = final_data[final_data.index==index]['original_title'].values[0]
  if (i<30):
    print(i,title_from_index)
    i+=1

Movies suggested for you : 

1 The Avengers
2 Avengers: Age of Ultron
3 Captain America: The Winter Soldier
4 Thor
5 Iron Man 2
6 Captain America: Civil War
7 Thor: The Dark World
8 Iron Man 3
9 Iron Man
10 X-Men
11 The Incredible Hulk
12 Captain America: The First Avenger
13 Ant-Man
14 The Dark Knight Rises
15 Guardians of the Galaxy
16 Superman
17 X-Men: First Class
18 Fantastic Four
19 Hulk
20 The Amazing Spider-Man 2
21 Man of Steel
22 Spider-Man 3
23 Spider-Man
24 4: Rise of the Silver Surfer
25 X-Men: Days of Future Past
26 Superhero Movie
27 X-Men: Apocalypse
28 Star Trek
29 Spider-Man 2


In [31]:
def recommend(movie_name):
  list_of_all_titles = final_data['original_title'].tolist()
  find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
  close_match = find_close_match[0]
  index_of_the_movie = final_data[final_data['original_title'] == close_match]['index'].values[0]

  similarity_score = list(enumerate(similarity[index_of_the_movie]))
  sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

  i = 1
  for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = final_data[final_data.index==index]['original_title'].values[0]
    if (i<30):
      print(title_from_index)
      i+=1


In [32]:
recommend('Avatar')

Avatar
Alien
Aliens
Alien³
Planet of the Apes
Treasure Planet
Home
Moonraker
Titan A.E.
Mission to Mars
Lifeforce
Armageddon
Star Trek Into Darkness
Soldier
The Thing
Zathura: A Space Adventure
Space Chimps
Silent Running
Aliens vs Predator: Requiem
Dune
Lockout
Predators
John Carter
Lost in Space
E.T. the Extra-Terrestrial
Spaceballs
Sunshine
Gravity
Gattaca


In [33]:
import pickle
pickle.dump(final_data,open('movie_list.pkl','wb'))